In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from scipy.stats import zscore
import plotly.express as px

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('Electric_Vehicles.csv')

In [ ]:
#Optimising memory by appling correct data types to features
df = df.astype({
    'User ID': 'str',
    'Vehicle Model': 'category',
    'Charging Station ID': 'str',
    'Charging Station Location': 'category',
    'Charging Start Time': 'datetime64[ns]',
    'Charging End Time': 'datetime64[ns]',
    'Time of Day': 'category',
    'Day of Week': 'category',
    'Charger Type': 'category',
    'User Type': 'category'
})

print(df.dtypes)


#Replace missing values with median
df[['Energy Consumed (kWh)', 'Charging Rate (kW)', 'Distance Driven (since last charge) (km)']] = df[['Energy Consumed (kWh)', 'Charging Rate (kW)', 'Distance Driven (since last charge) (km)']].fillna(df[['Energy Consumed (kWh)', 'Charging Rate (kW)', 'Distance Driven (since last charge) (km)']].median())
df.isnull().sum()

#### Logical inconsistencies check. 

In [ ]:
inconsistent_charge = df[df['State of Charge (Start %)'] > df['State of Charge (End %)']]
print("Inconsistent charges: ", len(inconsistent_charge))


print("\nInconsistent stations:")
inconsistent_stations = df.groupby('Charging Station ID')['Charging Station Location'].nunique()
inconsistent_stations = inconsistent_stations[inconsistent_stations > 1]
print(inconsistent_stations.head(100))  


# Check for negative values
print("\n\nRows with Negative Energy Consumed:")
print(df[df['Energy Consumed (kWh)'] < 0])

print("\n\nRows with Negative Distance Driven:")
print(df[df['Distance Driven (since last charge) (km)'] < 0])

#### One-hot encoding of categorical data. 

In [ ]:
df = pd.get_dummies(df, columns=['Vehicle Model', 'Charging Station Location', 'Time of Day', 'Day of Week', 'Charger Type', 'User Type'])

# Convert only one-hot encoded columns to integers
df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

print(df.dtypes)

In [26]:
df.columns

Index(['User ID', 'Battery Capacity (kWh)', 'Charging Station ID',
       'Charging Start Time', 'Charging End Time', 'Energy Consumed (kWh)',
       'Charging Duration (hours)', 'Charging Rate (kW)',
       'Charging Cost (USD)', 'State of Charge (Start %)',
       'State of Charge (End %)', 'Distance Driven (since last charge) (km)',
       'Temperature (°C)', 'Vehicle Age (years)', 'Vehicle Model_BMW i3',
       'Vehicle Model_Chevy Bolt', 'Vehicle Model_Hyundai Kona',
       'Vehicle Model_Nissan Leaf', 'Vehicle Model_Tesla Model 3',
       'Charging Station Location_Chicago',
       'Charging Station Location_Houston',
       'Charging Station Location_Los Angeles',
       'Charging Station Location_New York',
       'Charging Station Location_San Francisco', 'Time of Day_Afternoon',
       'Time of Day_Evening', 'Time of Day_Morning', 'Time of Day_Night',
       'Day of Week_Friday', 'Day of Week_Monday', 'Day of Week_Saturday',
       'Day of Week_Sunday', 'Day of Week_Thursday'

#### Feature engineering.

In [27]:
# Goal 1. Predict Optimal Charging Schedules to Reduce Urban Bottlenecks
df['Charging Hour'] = df['Charging Start Time'].dt.hour
df['Peak Hour'] = df['Charging Hour'].apply(lambda x: 1 if x in [6, 7, 8, 17, 18, 19] else 0)
df['Charging Duration during Peak'] = df['Charging Duration (hours)'] * df['Peak Hour']

# later, combine this state of charge difference with 'energy consumed (kWh) to find if ppl are charging to full capacity or not
df['Charge Difference'] = df['State of Charge (End %)'] - df['State of Charge (Start %)']

df['Energy Consumed per Hour'] = df.groupby('Charging Hour')['Energy Consumed (kWh)'].transform('mean')
df['Energy Consumed per Day of Week'] = df.groupby('Day of Week_Monday')['Energy Consumed (kWh)'].transform('mean')
df['Energy Consumed per Location'] = df.groupby('Charging Station Location_Chicago')['Energy Consumed (kWh)'].transform('mean')

df['Temperature Impact'] = df['Energy Consumed (kWh)'] * df['Temperature (°C)']

df['Peak Hour by Day'] = df['Peak Hour'] * (df['Day of Week_Monday'] +
                                            df['Day of Week_Tuesday'] +
                                            df['Day of Week_Wednesday'] +
                                            df['Day of Week_Thursday'] +
                                            df['Day of Week_Friday'] +
                                            df['Day of Week_Saturday'] +
                                            df['Day of Week_Sunday'])

df['Distance per Charging Duration'] = df['Distance Driven (since last charge) (km)'] / df['Charging Duration (hours)']

In [28]:
# Goal 2. Recommend Charger Type
# Energy consumption, duration, and cost by charger type
df['Energy Consumed per Charger Type'] = df.groupby('Charger Type_DC Fast Charger')['Energy Consumed (kWh)'].transform('mean')
df['Charging Duration per Charger Type'] = df.groupby('Charger Type_Level 1')['Charging Duration (hours)'].transform('mean')
df['Charging Cost per Charger Type'] = df.groupby('Charger Type_Level 2')['Charging Cost (USD)'].transform('mean')

# Energy consumption, duration, and cost by vehicle model
df['Energy Consumed per Vehicle Model'] = df.groupby('Vehicle Model_BMW i3')['Energy Consumed (kWh)'].transform('mean')
df['Charging Duration per Vehicle Model'] = df.groupby('Vehicle Model_Tesla Model 3')['Charging Duration (hours)'].transform('mean')
df['Charging Cost per Vehicle Model'] = df.groupby('Vehicle Model_Chevy Bolt')['Charging Cost (USD)'].transform('mean')
df['Charging Cost per Vehicle Model'] = df.groupby('Vehicle Model_Hyundai Kona')['Charging Cost (USD)'].transform('mean')
df['Charging Cost per Vehicle Model'] = df.groupby('Vehicle Model_Nissan Leaf')['Charging Cost (USD)'].transform('mean')

# Energy consumed and duration per user type
df['Energy Consumed per User Type'] = df.groupby('User Type_Casual Driver')['Energy Consumed (kWh)'].transform('mean')
df['Charging Duration per User Type'] = df.groupby('User Type_Commuter')['Charging Duration (hours)'].transform('mean')
df['Charging Cost per User Type'] = df.groupby('User Type_Long-Distance Traveler')['Charging Cost (USD)'].transform('mean')

# Interaction between charger type, vehicle model, and user type
df['Charger-Vehicle Interaction'] = df['Charger Type_DC Fast Charger'] * df['Vehicle Model_BMW i3']
df['Charger-User Interaction'] = df['Charger Type_Level 2'] * df['User Type_Commuter']
df['Vehicle-User Interaction'] = df['Vehicle Model_Tesla Model 3'] * df['User Type_Long-Distance Traveler']

# Additional aggregate features (optional)
df['Average Charging Cost per Location'] = df.groupby('Charging Station Location_Chicago')['Charging Cost (USD)'].transform('mean')
df['Average Energy Consumed per Location'] = df.groupby('Charging Station Location_New York')['Energy Consumed (kWh)'].transform('mean')

In [ ]:
# Goal 3. Identify Underutilized Stations

# Station Utilization: Count of charges per station
df['Station Utilization'] = df.groupby('Charging Station ID')['Charging Station ID'].transform('count')
# Energy consumed per station location
df['Energy Consumed per Station Location'] = df.groupby('Charging Station Location_Chicago')['Energy Consumed (kWh)'].transform('mean')
# Energy consumed by user type at location
df['Energy Consumed per User Type at Location'] = df.groupby(['Charging Station Location_Houston', 'User Type_Commuter'])['Energy Consumed (kWh)'].transform('mean')
# Charging duration per station location
df['Charging Duration per Location'] = df.groupby('Charging Station Location_Los Angeles')['Charging Duration (hours)'].transform('mean')
# Charging cost per station location
df['Charging Cost per Location'] = df.groupby('Charging Station Location_San Francisco')['Charging Cost (USD)'].transform('mean')
# Total charges per station
df['Total Charges per Station'] = df.groupby('Charging Station ID')['Charging Station ID'].transform('count')
# Energy consumed by time of day
df['Energy Consumed per Time of Day'] = df.groupby(['Charging Station Location_Chicago', 'Time of Day_Morning'])['Energy Consumed (kWh)'].transform('mean')
# Utilization rate (Energy consumed per charge)
df['Utilization Rate'] = df['Energy Consumed (kWh)'] / df['Station Utilization']
# Energy consumed per user type at station
df['Energy Consumed per User Type at Station'] = df.groupby(['Charging Station ID', 'User Type_Casual Driver'])['Energy Consumed (kWh)'].transform('mean')
# Total energy consumed per location
df['Total Energy Consumed per Location'] = df.groupby('Charging Station Location')['Energy Consumed (kWh)'].transform('sum')
# Total duration of charging per station
df['Total Duration per Station'] = df.groupby('Charging Station ID')['Charging Duration (hours)'].transform('sum')
# A charger with high utilisation can imply underserved area 
df['charger_availability_rate'] = df['Total Charges per Station'] / df['Total Slots at Station']



In [ ]:
# cont. Goal 3: Finding EV desert zones


# Geospatial Analysis:
#     To truly identify EV desert zones, you need to consider the geographic distribution of charging stations. A simple way to assess this is to analyze the distance between charging stations in different regions. Areas that are too far apart (e.g., >X miles/km) could be considered EV desert zones.
#     You could use geospatial clustering (like DBSCAN or KMeans) to identify regions with low station density.


# Population Density:
#     Cross-reference your data with population density. High-density areas with few charging stations could indicate EV deserts. Consider adding a feature based on the number of residents or cars per square kilometer in each area.


# Charging Station Availability:
#     Consider adding a feature for charging station availability, i.e., whether stations are consistently in use or whether they have free chargers available. If a station consistently shows high utilization or long waiting times, it may indicate an under-served zone where additional stations are needed.


# Traveling Patterns (Distance from Stations):
#     You could include the distance users travel to reach the charging station. If a station is located too far from a large number of users, it may point to an EV desert zone in that geographic region.


# Network Expansion Opportunity:
#     Use historical charging data and trends to predict where future demand is likely to grow. Under-served regions with growing populations or heavy commuter traffic could be potential EV desert zones for future network expansion.

In [ ]:
# Check the new features
pd.set_option("display.max_columns", None)
df.head()


In [ ]:
# # Goal 1: Predict Optimal Charging Schedules to Reduce Urban Bottlenecks and Minimize Grid Strain During Peak Hours

# df['Charging Start Hour'] = df['Charging Start Time'].dt.hour
# df['Charging Duration per Hour'] = df.groupby('Charging Start Hour')['Charging Duration (hours)'].transform('sum')
# df['Charging Duration per Hour'] = df.groupby('Charging Start Hour')['Charging Duration (hours)'].transform('sum')
# df['Charging Duration by Time of Day'] = df.groupby('Time of Day_Evening')['Charging Duration (hours)'].transform('mean')

# peak_hours = [16, 17, 18.5]  # Eg. if 4pm to 7.30pm
# df['Peak Hour'] = df['Charging Start Hour'].isin(peak_hours).astype(int)


# # Goal 2: Recommend Charger Type Based on Patterns of Energy Consumption, Duration, Cost, User Demographics, and Vehicle Model to Promote a Personalized EV Experience
# df['Energy Consumed per User Type'] = df.groupby('User Type_Commuter')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per User Type_Long-Distance Traveler'] = df.groupby('User Type_Long-Distance Traveler')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per User Type_Casual Driver'] = df.groupby('User Type_Casual Driver')['Energy Consumed (kWh)'].transform('mean')

# # Charging Cost per User Type
# df['Charging Cost per User Type_Commuter'] = df.groupby('User Type_Commuter')['Charging Cost (USD)'].transform('mean')
# df['Charging Cost per User Type_Long-Distance Traveler'] = df.groupby('User Type_Long-Distance Traveler')['Charging Cost (USD)'].transform('mean')
# df['Charging Cost per User Type_Casual Driver'] = df.groupby('User Type_Casual Driver')['Charging Cost (USD)'].transform('mean')

# # Charging Rate per vehicle model
# df['Charging Rate per Vehicle Model_BMW i3'] = df.groupby('Vehicle Model_BMW i3')['Charging Rate (kW)'].transform('mean')
# df['Charging Rate per Vehicle Model_Chevy Bolt'] = df.groupby('Vehicle Model_Chevy Bolt')['Charging Rate (kW)'].transform('mean')
# df['Charging Rate per Vehicle Model_Hyundai Kona'] = df.groupby('Vehicle Model_Hyundai Kona')['Charging Rate (kW)'].transform('mean')
# df['Charging Rate per Vehicle Model_Nissan Leaf'] = df.groupby('Vehicle Model_Nissan Leaf')['Charging Rate (kW)'].transform('mean')
# df['Charging Rate per Vehicle Model_Tesla Model 3'] = df.groupby('Vehicle Model_Tesla Model 3')['Charging Rate (kW)'].transform('mean')

# # Energy consumed per vehicle model
# df['Energy Consumed per Vehicle Model_BMW i3'] = df.groupby('Vehicle Model_BMW i3')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Vehicle Model_Chevy Bolt'] = df.groupby('Vehicle Model_Chevy Bolt')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Vehicle Model_Hyundai Kona'] = df.groupby('Vehicle Model_Hyundai Kona')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Vehicle Model_Nissan Leaf'] = df.groupby('Vehicle Model_Nissan Leaf')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Vehicle Model_Tesla Model 3'] = df.groupby('Vehicle Model_Tesla Model 3')['Energy Consumed (kWh)'].transform('mean')


# # Energy Consumed per Charger Type
# df['Energy Consumed per Charger Type_DC Fast Charger'] = df.groupby('Charger Type_DC Fast Charger')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Charger Type_Level 1'] = df.groupby('Charger Type_Level 1')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Charger Type_Level 2'] = df.groupby('Charger Type_Level 2')['Energy Consumed (kWh)'].transform('mean')

# df['Charging Cost per User Type'] = df.groupby('User Type_Commuter')['Charging Cost (USD)'].transform('mean')


# # Energy Consumed by Time of Day
# df['Energy Consumed per Time of Day_Afternoon'] = df.groupby('Time of Day_Afternoon')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Time of Day_Evening'] = df.groupby('Time of Day_Evening')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Time of Day_Morning'] = df.groupby('Time of Day_Morning')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Time of Day_Night'] = df.groupby('Time of Day_Night')['Energy Consumed (kWh)'].transform('mean')

# # Energy Consumed by Day of Week
# df['Energy Consumed per Day of Week_Friday'] = df.groupby('Day of Week_Friday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Monday'] = df.groupby('Day of Week_Monday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Saturday'] = df.groupby('Day of Week_Saturday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Sunday'] = df.groupby('Day of Week_Sunday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Thursday'] = df.groupby('Day of Week_Thursday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Tuesday'] = df.groupby('Day of Week_Tuesday')['Energy Consumed (kWh)'].transform('mean')
# df['Energy Consumed per Day of Week_Wednesday'] = df.groupby('Day of Week_Wednesday')['Energy Consumed (kWh)'].transform('mean')





# # Goal 3: Identify Underutilized Stations and Recommend Strategies to Boost Usage
# df['Station Utilization'] = df.groupby('Charging Station ID')['Charging Station ID'].transform('count')
# df['Cost per kWh'] = df['Charging Cost (USD)'] / df['Energy Consumed (kWh)']

# df['Location Popularity_Houston'] = df.groupby('Charging Station Location_Houston')['Charging Station Location_Houston'].transform('count')
# df['Location Popularity_Los Angeles'] = df.groupby('Charging Station Location_Los Angeles')['Charging Station Location_Los Angeles'].transform('count')
# df['Location Popularity_New York'] = df.groupby('Charging Station Location_New York')['Charging Station Location_New York'].transform('count')
# df['Location Popularity_San Francisco'] = df.groupby('Charging Station Location_San Francisco')['Charging Station Location_San Francisco'].transform('count')

# df['Charging Duration per Location_Houston'] = df.groupby('Charging Station Location_Houston')['Charging Duration (hours)'].transform('sum')
# df['Charging Duration per Location_Los Angeles'] = df.groupby('Charging Station Location_Los Angeles')['Charging Duration (hours)'].transform('sum')
# df['Charging Duration per Location_New York'] = df.groupby('Charging Station Location_New York')['Charging Duration (hours)'].transform('sum')
# df['Charging Duration per Location_San Francisco'] = df.groupby('Charging Station Location_San Francisco')['Charging Duration (hours)'].transform('sum')





# # MISC
# df['Is Weekend'] = df['Day of Week_Saturday'].astype(int) | df['Day of Week_Sunday'].astype(int)

# df['User-Vehicle Interaction'] = df['User Type_Commuter'].astype(str) + '-' + df['Vehicle Model_Chevy Bolt'].astype(str)
# df['User-Vehicle Interaction_Hyundai Kona'] = df['User Type_Commuter'].astype(str) + '-' + df['Vehicle Model_Hyundai Kona'].astype(str)
# df['User-Vehicle Interaction_Nissan Leaf'] = df['User Type_Commuter'].astype(str) + '-' + df['Vehicle Model_Nissan Leaf'].astype(str)
# df['User-Vehicle Interaction_Tesla Model 3'] = df['User Type_Commuter'].astype(str) + '-' + df['Vehicle Model_Tesla Model 3'].astype(str)


In [ ]:
df.columns

#### Correlation matrix

In [ ]:
# Filter out non-numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=['float64', 'int64', 'datetime64[ns]'])

# Calculate the correlation matrix on the numeric data
correlation_matrix = numeric_df.corr()

# better matrix via plotly
fig = px.imshow(correlation_matrix, text_auto=True, color_continuous_scale='Viridis')
fig.update_layout(width=1000, height=1000)
fig.show()


In [ ]:
print(df.isna().sum())  # Check for missing values
print(df.describe())  # Check for extreme values in numeric columns


#### Feature importance

In [ ]:
# # Split data into features (X) and target (y)
# x = df.drop(columns=['User ID'])
# y = df['User ID']

# # Train a RandomForest model
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(x, y)

# # Get feature importances
# importances = model.feature_importances_

# # Display feature importances
# feature_importance_df = pd.DataFrame({
#     'Feature': x.columns,
#     'Importance': importances
# })

# # Sort by importance
# feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
# print(feature_importance_df)